# Recommendation Systems, Vector Databases, and Music

![main](../images/main_pic.png)

## Table of Contents

1. Overview
2. The Challenge
3. Audio Data
    - Intro to Audio Data
    - Data Preparation
4. Vector Databases
    - What are they?
    - Why do we need them?
    - How can we use them?
    - Enter Qdrant
        - Getting Started
        - Adding Points
        - Payloads
        - Search
5. Models and Vector Representations
    - What role do Models play?
    - What are Embeddings?
    - Fine tunning Wav2Vec
    - Extracting Embeddings
6. Putting it all together
    - Basics of Recommender Systems
    - Loading up Qdrant
    - Building a UI
7. Final Thoughts

## 1. Overview

Vector databases are a relatively new way for interacting with abstract data representations derived from opaque machine learning models -- deep learning architectures being the most common ones. These representations are often called vectors or embeddings and they are a compressed version of the data used to train a machine learning model to accomplish a task (e.g., sentiment analysis, speech recognition, object detection, and many more).

Vector databases shine in many applications like semantic search and recommendation systems, and in this tutorial, we'll learn about how to build these two kinds of applications using [Qdrant](qdrant.tech), vector similarity search engine that provides a production-ready service with a convenient API to store, search, and manage points (i.e. vectors) with an additional payload.

Now, a tutorial is most useful when it involves a real use case, so let's go ahead and describe ours.

## 2. The Challenge

Building recommendation systems can be quite challenging. For starters, we never know apriori the needs and wants of the new customer of a store or the new user of a mobile app, and this makes is difficult to recommend, say, a toaster brand to someone searching for skillets at a store, or a Beatles' song to new users that just listened to a bachata by Romeo Santos and Drake.

The aforementioned problems -- "new user" == "no data" -- belong to the "cold start" family of problems in recommender systems and, while these problems might not go away anytime soon, there are ways to bypass them and serve relevant results from the get go, and one answer is vector databases. That said, that's what we will work on in this tutorial, we will build a music recommendation system on top of Qdrant.

Music data can be quite challenging to use given that most of it is copyright-protected so chances are you will need to find your data yourself to work on your use-case, use copyright-free music, or see if there is a non-conventional way to bypass the legal barriers around music data. That said, there's a latin music dataset in Kaggle called, [Tropical Genres Dataset](https://www.kaggle.com/datasets/carlossalazar65/tropical-genres-dataset). It contains 30-second clips of songs from the genres of bachata, salsa, merengue, cumbia, and vallenato. Some of these 30-second clips belong to the same songs and, while that is not ideal, it will work well for educational purposes.

Once you download it, you should see the following directories.

```sh
../data/ludwig_music_data
├── labels.json
├── mfccs
│   ├── blues
│   ├── ...
├── mp3
│   ├── blues
│   ├── classical
│   ├── electronic
│   ├── funk _ soul
│   ├── hip hop
│   ├── jazz
│   ├── latin
│   ├── pop
│   ├── reggae
│   └── rock
├── spectogram
│   └── spectogram
└── subgeneres.json
```

## 3. Audio Data

The data we'll use can be downloaded from Kaggle [here](https://www.kaggle.com/datasets/jorgeruizdev/ludwig-music-dataset-moods-and-subgenres?resource=download&select=labels.json), and it consists of the following pieces.
- `mp3/mp3`: two mp3 directories (unnecessarily) with audio files categorized by genre (e.g., latin, hip_hop, etc.). The name of each song is also the unique ID of the same. For simplicity make sure you move the files in the second mp3 directory up one level.
- `spectogram`: Mel Frequency Spectograms of each song saved as `.npy` files.
- `MFCCS`: This folder contains a .npy file with the extracted MFFCs of each song. Each `.npy` file contains around 10 MFFCs of 3s of duration.
- `labels.json`: metadata about each song including genre, artist, song, etc. We can use the unique ID to merge it with the main dataset.
- `subgenres.json`: metadata about the subgenre of each song. We can use the unique ID to merge it with the main dataset.

Once you download the data and unzip it inside your data directory, make sure you have the directory arranged in the following way to match this tutorial.

```sh
../data/ludwig_music_data
├── labels.json
├── mfccs
│   ├── blues
│   ├── ...
├── mp3
│   ├── blues
│   ├── classical
│   ├── electronic
│   ├── funk _ soul
│   ├── hip hop
│   ├── jazz
│   ├── latin
│   ├── pop
│   ├── reggae
│   └── rock
├── spectogram
│   └── spectogram
└── subgeneres.json
```

Please note that you can substitute this (12gb) dataset for another of your choosing, provided it is for a classification task, and follow the tutorial with barely any tweaks to the code.

Now that we know a bit about the challenge we will be tackling and the data we will be working with, let's talk about audio data in general to build an intuition on how things work.

### 3.1 Intro to Audio Data

Audio data refers to any type of sound that can be stored and transmitted in a digital format. This can include music, spoken words, and other types of sound recordings. In some ways, it is similar to how text can be stored and transmitted in a digital format. Hence, audio data is way of representing sound in a shape and form that can be processed and analyzed by computers. A wide range of applications use audio data and these include music production, telecommunications, and digital assistants like Siri and Alexa.

For data science use cases, we need to convert sound data to Mel spectrograms for use in machine learning models, the audio data is first divided into small segments. Each segment is then processed using a mathematical operation known as the Fourier transform, which breaks down the sound into its component frequencies. 

After the Fourier transform has been applied, mel-scale filter banks are used to group the frequencies into a set of bands that more closely match the human auditory system's perception of sound. These filter banks essentially amplify some frequency ranges while reducing others. This process results in a set of values for each segment of audio, which can be arranged to form a spectrogram.

Mel-spectrograms are useful in machine learning applications as they provide a way to represent audio data in a format that can be easily processed and analyzed by computer models. The mel-spectrograms can be used as inputs to machine learning models, which can then learn to recognize patterns and classify different types of sounds.

Now that we know a little bit about audio data, let's examine a sample to get an intuition for how the process works.

Before you run any line of code, make sure you create a virtual environment with the following packages.

```bash
# with conda
mamba env create -n my_env python=3.10
mamba activate my_env

# or with virtualenv
python -m venv venv
source venv/bin/activate

# install packages
pip install qdrant-client transformers datasets pandas numpy streamlit torch
```

In [1]:
from IPython.display import Audio as player
import numpy as np

In [2]:
from huggingface_hub import HfApi, login

In [3]:
login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/ramonperez/.cache/huggingface/token
Login successful


### 3.2 Data Prep

In [6]:
from datasets import load_dataset, Audio
import torch

In [9]:
# music_data = load_dataset("ramonpzg/latin_music", split="train")
# music_data

In [10]:
music_data = load_dataset(
    "audiofolder", data_dir="../data/latin_music/Audios/", split="train"
).shuffle(seed=42).select(range(200))

music_data

Resolving data files:   0%|          | 0/900 [00:00<?, ?it/s]

Found cached dataset audiofolder (/home/ramonperez/.cache/huggingface/datasets/audiofolder/default-a4fb2cc2c8fd3974/0.0.0/6cbdd16f8688354c63b4e2a36e1585d05de285023ee6443ffd71c4182055c0fc)


Dataset({
    features: ['audio', 'label'],
    num_rows: 200
})

In [11]:
music_data[0]

{'audio': {'path': '/home/ramonperez/Tresors/datascience/challenges/qdrant_chl/data/latin_music/Audios/Merengue/merengue0120.mp3',
  'array': array([0.        , 0.        , 0.        , ..., 0.5292567 , 0.52702796,
         0.4594567 ], dtype=float32),
  'sampling_rate': 44100},
 'label': 1}

In [12]:
music_data = music_data.cast_column("audio", Audio(sampling_rate=16_000))

In [13]:
paths = [music_data[i]['audio']['path'] for i in range(len(music_data))]

In [9]:
music_data = music_data.add_column("paths", paths)

In [12]:
labels = music_data.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

num_labels = len(id2label)

In [13]:
def get_names(label_num):
    return id2label[str(label_num)]

label_names = list(map(get_names, music_data['label']))
music_data = music_data.add_column("label_names", label_names)
music_data[-10]

{'audio': {'path': '/home/ramonperez/Tresors/datascience/challenges/qdrant_chl/data/latin_music/Audios/Salsa/salsa0290.mp3',
  'array': array([0.        , 0.        , 0.        , ..., 0.16269699, 0.19887206,
         0.07940404], dtype=float32),
  'sampling_rate': 16000},
 'label': 2,
 'paths': '/home/ramonperez/Tresors/datascience/challenges/qdrant_chl/data/latin_music/Audios/Salsa/salsa0290.mp3',
 'label_names': 'Salsa'}

In [10]:
music_data.unique("label")

[0, 1, 2]

Time for the labels.

In [ ]:
import pandas as pd

In [ ]:
labels = pd.read_json("../data/ludwig_music_data/labels.json")

In [ ]:
from random import choice
labels['tracks'].iloc[choice(range(200))]

In [ ]:
def get_metadata(x):
    try:
        artist = list(x['artist'].values())[0]
        genre = list(x['genre'].values())[0]
        name = list(x['name'].values())[0]
    except:
        artist = "Unknown"
        genre = "Unknown"
        name = "Unknown"
    return pd.Series([artist, genre, name], index=['artist', 'genre', 'name'])

In [ ]:
clean_labels = labels['tracks'].apply(get_metadata).reset_index()
clean_labels.head()

In [ ]:
clean_labels.name.value_counts()

In [ ]:
music_data = music_data.to_pandas().merge(
    right=clean_labels, left_on='idx', right_on='index', how="left"
).drop("index", axis=1)

music_data.head()

In [ ]:
from datasets import Dataset

In [ ]:
music_data = Dataset.from_pandas(music_data, preserve_index=False)
music_data = music_data.cast_column('audio', Audio(sampling_rate=16_000))
music_data

In [61]:
music_data.remove_columns(['paths', 'label_names']).push_to_hub("ramonpzg/latin_music", max_shard_size="1GB")

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/450 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 music_data.remove_columns(['paths', 'label_names']).push_to_hub("ramonpzg/latin_music",      │
│   2                                                                                              │
│                                                                                                  │
│ /home/ramonperez/anaconda/envs/vdb_audio/lib/python3.10/site-packages/datasets/arrow_dataset.py: │
│ 5245 in push_to_hub                                                                              │
│                                                                                                  │
│   5242 │   │   │   )                                                                             │
│   5243 │   │   │   dataset_metadata = DatasetMetadata.from_readme(Path(dataset_readme_path))     │
│   5244 │   │   │   dataset_infos: DatasetInfosDict = DatasetInfosDict.from_metadata(dataset_met  │
│ ❱ 5245 │   │   │   repo_info = dataset_infos[next(iter(dataset_infos))]                          │
│   5246 │   │   # get the deprecated dataset_infos.json to uodate them                            │
│   5247 │   │   elif config.DATASETDICT_INFOS_FILENAME in repo_files:                             │
│   5248 │   │   │   dataset_metadata = DatasetMetadata()                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
StopIteration

## 4. Vector Databases

### 4.1 What Are They?

A vector database is a type of database designed to store and query high-dimensional vectors efficiently. In traditional databases, data is typically organized in rows and columns, and queries are performed based on the values in those columns. However, in certain applications, such as machine learning, image recognition, natural language processing, and recommendation systems, data is often represented as vectors in a high-dimensional space.

A vector in this context is a mathematical representation of an object or data point, where each element of the vector corresponds to a specific feature or attribute of the object. For example, in an image recognition system, a vector could represent an image, with each element of the vector representing a pixel value or a descriptor of that pixel.

Vector databases are optimized for storing and querying these high-dimensional vectors efficiently, often using specialized data structures and indexing techniques. They enable fast similarity searches, allowing users to find vectors that are similar to a given query vector based on some distance metric, such as Euclidean distance or cosine similarity.

### 4.2 Why do we need them?

Vector databases play a crucial role in various applications that require similarity search, such as recommendation systems, content-based image retrieval, and personalized search. By leveraging efficient indexing and search techniques, vector databases enable faster and more accurate retrieval of similar vectors, enabling advanced data analysis and decision-making.

### 4.3 How can we use them?

To get started using vector databases, it is important that we understand the basics of how vectors are generated. We do not need to be a data scientist or a machine learning engineer, but some familiarity with machine learning (and/or the ability to follow a comprehensive tutorial) would be enough to help us get our hands on some vectors. 

Once we train an algorithm, most-likely a deep learning one, you will have at your disposal the vector representation of the data used to train the model, and the model itself to transform new data into vectors. Note that, even if you trained your algorithm for a particular task, what you need is the structured representation of your unstructured data (e.g., images, audio, text, etc.) rather than the predictive function itself. The goal is to compare the new sample to existing ones to get the most similar results on the fly.

The next step is to pick a vector database that aligns with what you, your team, or company is trying to do (e.g., recommendation system, semantic search, ranking, etc.). There are many solutions out and they all differ in terms of ease of use (i.e., how many more tools you need in order to get started), distribution (e.g. SaaS vs Open Source), accessibility (i.e., can you find new creative ways to use the low-level parts of the database?), and coverage (e.g., similarity metrics available and capabilities such as payloads).

Once you have you vector database set up, and have gotten familiarized with the API, start adding vector and create a simple UI to test your use case, for example, use streamlit or nicegui to put together a nice looking prototype that you can share and get feedback from. If everyone is happy with the results, start drafting a plan for how you will productionize, maintain, and monitor your database, as well as what are the steps to provide access to the database to developers or other data professionals working on recommendation systems.

With that out of the way, let's get started with vector databases using the fastest growing open source tool in the market, Qdrant! 😎

### 4.3 Enter Qdrant

![qdrant](../images/qdrant_overview.png)

#### 4.3.1 Getting Started

In order to get started with Qdrant, we can either pull the latest docker image using `docker pull qdrant/qdrant`, and then run it with,
```bash
docker run -p 6333:6333 \
    -v $(pwd)/qdrant_storage:/qdrant/storage \
    qdrant/qdrant
```
or we can use its Python client and use in in-memory functionality to play around with it. We will opt for the former in this tutorial but feel free to use the in-memory version if you prefer.

We'll start by instantiating our client using `host="localhost"` and `port=6333` as opposed to `host=":memory:"` for the in-memory option.

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http import models

In [ ]:
client = QdrantClient(host="localhost", port=6333)
client

In more OLTP or OLAP databases we call specific logical bundles of data `Tables`, but in vector databases, we refer to these collections of vectors as `collections`. In the same way in which we can create many tables in a database, we can create many collections in a vector-based db using a client. The key difference to note is that when we create a collection, we need to specify the width of the collection with the parameter `size=...`, and the similarity metric `distance=...` (which can be changed later on).

Let's create our first collection.

In [ ]:
first_collection = client.create_collection(
    collection_name="my_first_collection",
    vectors_config=models.VectorParams(size=100, distance=models.Distance.COSINE)
)

In [ ]:
# we can check that our collection was indeed created with
client.get_collections()

There's a couple of things to notice from what we have done.
- The first is that when we initiated our docker image, we created a local directory, `qdrant_storage`, where all of our collections, plus their metadata, will be saved at. You can have a look at that directory in a *nix system with `tree qdrant_storage -L 2`. You should see the following.
    ```bash
    qdrant_storage
    ├── aliases
    │   └── data.json
    ├── collections
    │   └── my_first_collection
    └── raft_state
    ```
- The second is that we used `client.create_collection` and this command can only be used once per collection. To recreate the collection with new parameters and the like, we would use `client.recreate_collection` instead.

Now that we know how to create collections, let's create a bit of fake data and add some vectors to our collection.

#### 4.3.2 Adding Points

The points are the central entity that Qdrant operates with, and these points contain records consisting of a vector, an optional id and an optional payload (which we'll talk more about in the next section).

The optional id can be represented by unassigned integers or UUIDs. We are going a range of numbers for this.

In [ ]:
data = np.random.uniform(low=-1.0, high=1.0, size=(1_000, 100))
data

In [ ]:
index = list(range(len(data)))
index[-10:]

In [ ]:
client.upsert(
    collection_name="my_first_collection",
    points=models.Batch(
        ids=index,
        vectors=data.tolist()
    )
)

We can retrieve specific points based on their ID (for example, artist X with ID 1000) and get some additional information from that result.

In [ ]:
client.retrieve(
    collection_name="my_first_collection",
    ids=[100],
    with_vectors=True # we can turn this on and off depending on our needs
)

We can also update our collection one point at a time, for example, as new data comes in.

In [ ]:
def create_song():
    return np.random.uniform(low=-1.0, high=1.0, size=100).tolist()

In [ ]:
client.upsert(
    collection_name="my_first_collection",
    points=[
        models.PointStruct(
            id=1000,
            vector=create_song(),
        )
    ]
)

We can also delete it in a straightforward fashion.

In [ ]:
client.count(
    collection_name="my_first_collection", 
    exact=True,
)

In [ ]:
client.delete(
    collection_name="my_first_collection",
    points_selector=models.PointIdsList(
        points=[1000],
    ),
)

In [ ]:
client.count(
    collection_name="my_first_collection", 
    exact=True,
)

#### 4.3.3 Payloads

Qdrant has incredible features on top of speed and reliability, and one of its most useful ones is without a doubt the ability to store additional information along with vectors. In Qdrant terminology, this information is considered a payload and it is represented as a JSON file. In addition, not only can you get this information back when you search in the database, but you can also filter your search by the parameters in the payload, and we'll see how in a second.

Imagine the fake vectors we created actually represented a song. If we were building a recommender system for songs then, naturally, the things we would want to get back would be the song itself, the artist, maybe the genre, and so on.

What we'll do here is to take advantage of a Python package call `faker` and create a bit of information to add to our payload.

In [ ]:
from faker import Faker

In [ ]:
fake_something = Faker()
fake_something.name()

In [ ]:
payload = []

for i in range(len(data)):
    payload.append(
        {
            "artist": fake_something.name(),
            "song": " ".join(fake_something.words()),
            "url_song": fake_something.url(),
            "year": fake_something.year(),
            "country": fake_something.country()
        }
    )

payload[:3]

In [ ]:
client.upsert(
    collection_name="my_first_collection",
    points=models.Batch(
        ids=index,
        vectors=data.tolist(),
        payloads=payload
    )
)

In [ ]:
resutls = client.retrieve(
    collection_name="my_first_collection",
    ids=[10, 50, 100, 500],
    with_vectors=False
)
resutls

In [ ]:
resutls[0].payload

In [ ]:
# client.clear_payload(
#     collection_name="my_first_collection",
#     points_selector=models.PointIdsList(
#         points=index,
#     )
# )

#### 4.3.4 Search

Now that we have our vectors with an ID and a payload, we can explore a few of ways in which we can search for content when, in our use case, new music gets selected. Let's check it out.

Say, for example, that a new song comes in and our model immediately transforms it into a vector.

In [ ]:
living_la_vida_loca = create_song()

In [ ]:
client.search(
    collection_name="my_first_collection",
    query_vector=living_la_vida_loca,
    limit=10
)

Now imagine that we only want Australian songs recommended to us.

In [ ]:
aussie_songs = models.Filter(
    must=[models.FieldCondition(key="country", match=models.MatchValue(value="Australia"))]
)

In [ ]:
client.search(
    collection_name="my_first_collection",
    query_vector=living_la_vida_loca,
    query_filter=aussie_songs,
    limit=5
)

Lastly, say we want aussie songs but we don't care how new or old these songs are.

In [ ]:
client.search(
    collection_name="my_first_collection",
    query_vector=living_la_vida_loca,
    query_filter=aussie_songs,
    with_payload=models.PayloadSelectorExclude(exclude=["year"]),
    limit=5
)

As you can see, the possibilities are endless.

## 5. Transformers and Embeddings

In the context of audio data, embeddings and transformers are used to process the sound waves and extract features that are useful for training machine learning models.

### 5.1 What are transformers?

Transformers are a type of neural network used for natural language processing, but they can also be used for processing audio data by breaking the sound waves into smaller parts and learning how those parts fit together to form meaning.

### 5.2 What are Embeddings?

Embeddings are a way of representing audio data as vectors or numbers, which makes it easier for machine learning algorithms to process and analyze them.

### 5.3 Fine tunning Wav2Vec

In [62]:
import torch
# from datasets import Audio
from transformers import AutoModel, AutoFeatureExtractor

In [63]:
from IPython.display import Audio as player

In [64]:
sample = music_data[101]['audio']['array']
player(sample, rate=16_000)

In [65]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [66]:
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

In [81]:
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
inputs = feature_extractor(
    sample, sampling_rate=feature_extractor.sampling_rate, 
    return_tensors="pt", padding=True, return_attention_mask=True, max_length=16_000, truncation=True
).to(device)

/home/ramonperez/anaconda/envs/vdb_audio/lib/python3.10/site-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [83]:
inputs

{'input_values': tensor([[0.0042, 0.0042, 0.0042,  ..., 0.7640, 0.6211, 0.8564]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0', dtype=torch.int32)}

To download and instantiate a pre-trained model, we will use the AutoModel class from transformers.

Note that there are other ways of doing the same.
```python
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base")
```

In [29]:
from transformers import Wav2Vec2Processor

In [47]:
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base")

/home/ramonperez/anaconda/envs/vdb_audio/lib/python3.10/site-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [82]:
model = AutoModel.from_pretrained(
    "facebook/wav2vec2-base",# num_labels=num_labels, label2id=label2id, id2label=id2label
).to(device)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['quantizer.weight_proj.weight', 'project_hid.weight', 'quantizer.weight_proj.bias', 'project_hid.bias', 'project_q.weight', 'project_q.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 5.4 Extracting Embeddings

In [85]:
with torch.no_grad():
    embs = model(inputs.input_values, attention_mask=inputs.attention_mask).last_hidden_state
embs.size()

torch.Size([1, 49, 768])

In [96]:
def embed_audio(batch):
    audio_arrays = [x["array"] for x in batch["audio"]]

    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, 
        return_tensors="pt", padding=True, return_attention_mask=True, max_length=16_000, truncation=True
    ).to(device)

    with torch.no_grad():
        model_output = model(inputs.input_values, attention_mask=inputs.attention_mask).last_hidden_state

    pooled_embeds = torch.mean(model_output[0], axis=1)
    
    return {"embedding": pooled_embeds.cpu().numpy()}

In [97]:
data_w_embs = music_data.map(embed_audio, batched=True, batch_size=8)

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 data_w_embs = music_data.map(embed_audio, batched=True, batch_size=8)                        │
│   2                                                                                              │
│                                                                                                  │
│ /home/ramonperez/anaconda/envs/vdb_audio/lib/python3.10/site-packages/datasets/arrow_dataset.py: │
│ 563 in wrapper                                                                                   │
│                                                                                                  │
│    560 │   │   else:                                                                             │
│    561 │   │   │   self: "Dataset" = kwargs.pop("self")                                          │
│    562 │   │   # apply actual function                                                           │
│ ❱  563 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    564 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    565 │   │   for dataset in datasets:                                                          │
│    566 │   │   │   # Remove task templates if a column mapping of the template is no longer val  │
│                                                                                                  │
│ /home/ramonperez/anaconda/envs/vdb_audio/lib/python3.10/site-packages/datasets/arrow_dataset.py: │
│ 528 in wrapper                                                                                   │
│                                                                                                  │
│    525 │   │   │   "output_all_columns": self._output_all_columns,                               │
│    526 │   │   }                                                                                 │
│    527 │   │   # apply actual function                                                           │
│ ❱  528 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    529 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    530 │   │   # re-apply format to the output                                                   │
│    531 │   │   for dataset in datasets:                                                          │
│                                                                                                  │
│ /home/ramonperez/anaconda/envs/vdb_audio/lib/python3.10/site-packages/datasets/arrow_dataset.py: │
│ 2953 in map                                                                                      │
│                                                                                                  │
│   2950 │   │   │   │   │   leave=False,                                                          │
│   2951 │   │   │   │   │   desc=desc or "Map",                                                   │
│   2952 │   │   │   │   ) as pbar:                                                                │
│ ❱ 2953 │   │   │   │   │   for rank, done, content in Dataset._map_single(**dataset_kwargs):     │
│   2954 │   │   │   │   │   │   if done:                                                          │
│   2955 │   │   │   │   │   │   │   shards_done += 1                                              │
│   2956 │   │   │   │   │   │   │   logger.debug(f"Finished processing shard number {rank} of {n  │
│                                                                                                  │
│ /home/ramonperez/anaconda/envs/vdb_audio/lib/python3.10/sit

In [80]:
np.save("../data/recsys_2/music_embs", np.array(embeddings['embedding']), allow_pickle=False)

In [ ]:
def get_features(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    return feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, return_tensors="pt",
        max_length=16000, truncation=True, padding=True, return_attention_mask=True,
    ).to(device)

In [ ]:
music_data = music_data.map(get_features, batched=True, batch_size=50)
music_data

In [ ]:
music_data.set_format("torch", columns=["genre", "input_values"])

In [99]:
feature_extractor.model_input_names

['input_values', 'attention_mask']

In [ ]:
def extract_hidden_states(batch):
    inputs = {k: v.to(device) for k, v in batch.items() if k in feature_extractor.model_input_names}
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    return {"embeddings": last_hidden_state[:, 0].cpu().numpy()}

In [ ]:
hidden_state = music_data.map(extract_hidden_states, batched=True, batch_size=50)
hidden_state

In [ ]:
np.array(hidden_state["hidden_state"]).shape

In [ ]:
np.array(hidden_state["hidden_state"])[:2, :]

In [ ]:
np.save(
    '../data/recsys_1/vectors.npy', 
    np.array(hidden_state["hidden_state"]), 
    allow_pickle=False
)

Embeddings and transformers are tools used to extract important information from audio data and make it easier for computers to understand and work with that information. They are used in a wide range of applications, from speech recognition to music analysis.

In [ ]:
hidden_state.select_columns(
    ["genre", "artist", 'name', 'audio_path']
).to_pandas().to_json("../data/recsys_1/payload.json")

## 6. Putting it All Together

If you are using a notebook, this would be a good time to restart it.

In [ ]:
import pandas as pd
import torchaudio, torch
from qdrant_client import QdrantClient
from qdrant_client.http import models
import numpy as np
from transformers import AutoModel, AutoFeatureExtractor
from IPython.display import Audio as player

### 6.1 Basics of Recommender Systems

### 6.2 Loading Up Qdrant

In [ ]:
payload = pd.read_json("../data/recsys_1/payload.json", orient="records").to_dict(orient="records")
payload[0]

In [ ]:
vectors = np.load("../data/recsys_1/vectors.npy")
vectors.shape

In [ ]:
client = QdrantClient("localhost", port=6333)

In [ ]:
client.recreate_collection(
    collection_name="music_recsys",
    vectors_config=models.VectorParams(size=768, distance=models.Distance.COSINE),
)

In [ ]:
collection_info = client.get_collection(collection_name="music_recsys")
collection_info

In [ ]:
from qdrant_client.http.models import CollectionStatus

assert collection_info.status == CollectionStatus.GREEN
assert collection_info.vectors_count == 0

In [ ]:
client.upsert(
    collection_name="music_recsys",
    points=models.Batch(
        ids=list(range(len(vectors))),
        vectors=vectors.tolist(),
        payloads=payload
    )
)

In [ ]:
new_sample, sampling_rate = torchaudio.load("../data/ludwig_music_data/mp3/latin/0A6nzrnr0lUreHRTx3fRru.mp3")
new_sample

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")
model = AutoModel.from_pretrained("facebook/wav2vec2-base")#.to(device)

In [ ]:
new_input = feature_extractor(
    new_sample[0], sampling_rate=feature_extractor.sampling_rate, return_tensors="pt", padding=True
)#.to(device)

In [ ]:
with torch.no_grad():
    vectr = model(**new_input).last_hidden_state[:, 0].cpu().numpy()

In [ ]:
vectr = vectr[0,].cpu().numpy()

In [ ]:
player(new_sample[1], rate=44100)

In [ ]:
results2 = client.search(
    collection_name="music_recsys",
    query_vector=vectr.tolist(),
    limit=10, 
    # with_vectors=True
)
results2

### 6.3 Building a UI

In [ ]:
%%writefile recsys_app.py

from transformers import AutoModel, AutoFeatureExtractor
from qdrant_client import QdrantClient
import torch, torchaudio
import streamlit as st

st.title("Music Recommendation App")
st.markdown("Upload your favorite songs and get a list of recommendations from our database of music.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained('models/my_model').to(device)
feature_extractor = AutoFeatureExtractor.from_pretrained("models/my_model")
client = QdrantClient("localhost", port=6333)

music_file = st.file_uploader(label="📀 Music file 🎸",)

if music_file:
    st.audio(music_file)

    waveform, sample_rate = torchaudio.load(music_file)
    inputs = feature_extractor(
        waveform[0], sampling_rate=feature_extractor.sampling_rate, 
        return_tensors="pt", max_length=16000, truncation=True
    ).to(device)

    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state[:, 0]
    vectr = last_hidden_state.cpu().numpy()[0, :]

    st.markdown("## Real Recommendations")
    results = client.search(collection_name="music_recsys", query_vector=vectr, limit=4)
    col1, col2 = st.columns(2)

    with col1:
        st.header(f"Genre: {results[0].payload['genre']}")
        st.subheader(f"Artist: {results[0].payload['artist']}")
        # st.audio(results[0].payload["audio_path"])
        
        st.header(f"Genre: {results[1].payload['genre']}")
        st.subheader(f"Artist: {results[1].payload['artist']}")
        # st.audio(results[1].payload["audio_path"])

    with col2:
        st.header(f"Genre: {results[2].payload['genre']}")
        st.subheader(f"Artist: {results[2].payload['artist']}")
        # st.audio(results[5].payload["audio_path"])
        
        st.header(f"Genre: {results[3].payload['genre']}")
        st.subheader(f"Artist: {results[3].payload['artist']}")

In [ ]:
!streamlit run recsys_app.py

## 7. Final Thoughts